In [ ]:
#############################################################
# Author(s): Debaditya, Anwesha, Anna                       #
#############################################################

In [ ]:
#Download the data files.

import os, requests

fname = []
for j in range(3):
    fname.append('steinmetz_part%d.npz'%j)
url = ["https://osf.io/agvxh/download"]
url.append("https://osf.io/uv3mw/download")
url.append("https://osf.io/ehmw2/download")

for j in range(len(url)):
    if not os.path.isfile(fname[j]):
        try:
            r = requests.get(url[j])
        except requests.ConnectionError:
            print("!!! Failed to download data !!!")
        else:
            if r.status_code != requests.codes.ok:
                print("!!! Failed to download data !!!")
            else:
                with open(fname[j], "wb") as fid:
                    fid.write(r.content)

In [ ]:
#Setup dependencies

#Dependencies:
#numpy
#matplotlib

import numpy as np
from matplotlib import rcParams 
from matplotlib import pyplot as plt
import matplotlib.gridspec as gridspec

rcParams['figure.figsize'] = [20, 10]
rcParams['axes.spines.top'] = False
rcParams['axes.spines.right'] = False
rcParams['figure.autolayout'] = True

In [ ]:
#Groupings of brain regions

regions = ["vis ctx", "thal", "hipp", "other ctx", "midbrain", "basal ganglia", "cortical subplate", "other"]
brain_groups = [["VISa", "VISam", "VISl", "VISp", "VISpm", "VISrl"], # visual cortex
                ["CL", "LD", "LGd", "LH", "LP", "MD", "MG", "PO", "POL", "PT", "RT", "SPF", "TH", "VAL", "VPL", "VPM"], # thalamus
                ["CA", "CA1", "CA2", "CA3", "DG", "SUB", "POST"], # hippocampal
                ["ACA", "AUD", "COA", "DP", "ILA", "MOp", "MOs", "OLF", "ORB", "ORBm", "PIR", "PL", "SSp", "SSs", "RSP"," TT"], # non-visual cortex
                ["APN", "IC", "MB", "MRN", "NB", "PAG", "RN", "SCs", "SCm", "SCig", "SCsg", "ZI"], # midbrain
                ["ACB", "CP", "GPe", "LS", "LSc", "LSr", "MS", "OT", "SNr", "SI"], # basal ganglia 
                ["BLA", "BMA", "EP", "EPd", "MEA"] # cortical subplate
                ]
n_regions = len(regions)

In [ ]:
#Load the data

def load_alldat():
    alldat = np.array([])
    for j in range(len(fname)):
        alldat = np.hstack((alldat, np.load('steinmetz_part%d.npz'%j, allow_pickle=True)['dat']))
    return alldat

alldat = load_alldat()

In [ ]:
#Print the keys

print("The keys in this dataset are:\n----")
for key in alldat[0].keys():
    print(key)

In [ ]:
#Print information about brain area in mice:

for idx, dat in enumerate(alldat):
    print(idx,dat['mouse_name'],dat['date_exp'],np.unique(dat['brain_area']))

In [ ]:
#Select session

dat = alldat[11]
dt = dat['bin_size']

In [ ]:
#Generate some metadata for the neuron regions.

def generate_metadata(dat):
    n_neurons = (len(dat['brain_area']))
    n_regions = len(regions)
    region_index = np.ones(n_neurons)*n_regions
    group_index = np.ones(n_neurons)*n_regions
    for region in range(len(regions)-1):
        region_index[np.isin(dat['brain_area'], brain_groups[region])] = region
        for group in range(len(brain_groups[region])):
            group_index[np.isin(dat['brain_area'],brain_groups[region][group])] = group
    return n_neurons, region_index, group_index

In [ ]:
#Define function to print plots

def pcolormesh_with_means(region,spikes,session):
    for group in range(len(brain_groups[region])):
        if np.sum(np.logical_and(region_index==region,group_index == group)):
            print(np.shape(spikes[:][np.logical_and(region_index==region,group_index == group),:]))
            
            fig = plt.figure(constrained_layout=False)
            fig.suptitle("Avg Spike rate of all neurons in region "+brain_groups[region][group]+" for session "+str(session))
            gs = fig.add_gridspec(4, 5)
            
            fig_raster= fig.add_subplot(gs[:3, :4])
            raster_plot = fig_raster.pcolormesh(np.sum(spikes[:][np.logical_and(region_index==region,group_index == group),:],axis=0))
            fig_raster.set_ylabel("Trials")
            fig_raster.set_xlabel("Binned Time (10ms)")
                        
            fig_avg_trial = fig.add_subplot(gs[3,:4], sharex=fig_raster)
            fig_avg_trial.plot(np.mean(spikes[:][np.logical_and(region_index==region,group_index == group),:],axis=(0,1))/dt)
            fig_avg_trial.set_ylabel("Mean FR(Hz)")
            fig_avg_trial.set_xlabel("Binned Time (10ms)")
            fig_avg_trial.set_title("Mean Firing Rate Across Trials")
            
            fig_avg_time = fig.add_subplot(gs[:3,4],sharey=fig_raster)
            fig_avg_time.plot(np.mean(spikes[:][np.logical_and(region_index==region,group_index == group),:],axis=(0,2))/dt,range(np.shape(spikes)[1]))
            fig_avg_time.set_ylabel("Trials")
            fig_avg_time.set_xlabel("Mean FR (Hz)")
            fig_avg_time.set_title("Mean Firing in Trial")
            
            plt.show()


In [ ]:
#Print all plots

n_regions = len(regions)
for session, dat in enumerate(alldat):
    n_neurons, region_index, group_index = generate_metadata(dat)
    spikes = dat['spks']
    for region in range(n_regions-1):
        pcolormesh_with_means(region,spikes,session)
        

In [ ]:
#Normalize spikes

print(np.shape(dat['spks']))
print(np.shape(dat['spks_passive']))
spikes = dat['spks']-np.mean(dat['spks_passive'],axis=(1,2))[0]

In [ ]:
#Define function to print all mean plots.
for session in range(39):
    dat = alldat[session]
    spikes = dat['spks']-np.mean(dat['spks_passive'],axis=(1,2))[0]
    for idx, region in enumerate(np.unique(dat['brain_area'])):
        if np.mod(idx,3)==0:
            fig, axs = plt.subplots(3)
            axs[0].plot(np.mean(spikes[:][dat['brain_area']==region,:],axis=(0,1))/dt,label=str(session)+" "+region)
            axs[0].set_xlabel('Time Bins (10ms)')
            axs[0].set_ylabel('Mean Firing Rate Across Trials (Hz)')
            axs[0].legend()
        if np.mod(idx,3)==1:
            axs[1].plot(np.mean(spikes[:][dat['brain_area']==region,:],axis=(0,1))/dt,label=str(session)+" "+region)
            axs[1].set_xlabel('Time Bins (10ms)')
            axs[1].set_ylabel('Mean Firing Rate Across Trials (Hz)')
            axs[1].legend()
        if np.mod(idx,3)==2:
            axs[2].plot(np.mean(spikes[:][dat['brain_area']==region,:],axis=(0,1))/dt,label=str(session)+" "+region)
            axs[2].set_xlabel('Time Bins (10ms)')
            axs[2].set_ylabel('Mean Firing Rate Across Trials (Hz)')
            axs[2].legend()
            plt.show()
    # plt.show() 

In [ ]:
#Define function to print plots

def pcolormesh_with_single_neuron(region,spikes,session,neuron):
    for group in range(len(brain_groups[region])):
        if np.sum(np.logical_and(region_index==region,group_index == group)):
            print(np.shape(spikes[:][np.logical_and(region_index==region,group_index == group),:]))
            
            fig = plt.figure(constrained_layout=False)
            fig.suptitle("Avg Spike rate of a single neurons in region "+brain_groups[region][group]+" for session "+str(session))
            gs = fig.add_gridspec(4, 5)
            
            fig_raster= fig.add_subplot(gs[:3, :4])
            t_spikes = spikes[:][np.logical_and(region_index==region,group_index == group),:]
            raster_plot = fig_raster.pcolormesh(t_spikes[:][neuron,:])
            fig_raster.set_ylabel("Trials")
            fig_raster.set_xlabel("Binned Time (10ms)")
                        
            fig_avg_trial = fig.add_subplot(gs[3,:4], sharex=fig_raster)
            fig_avg_trial.plot(np.mean(t_spikes[:][neuron,:],axis=(0))/dt)
            fig_avg_trial.set_ylabel("Mean FR(Hz)")
            fig_avg_trial.set_xlabel("Binned Time (10ms)")
            fig_avg_trial.set_title("Mean Firing Rate Across Trials")
            
            fig_avg_time = fig.add_subplot(gs[:3,4],sharey=fig_raster)
            fig_avg_time.plot(np.mean(t_spikes[:][neuron,:],axis=(1))/dt,range(np.shape(t_spikes)[1]))
            fig_avg_time.set_ylabel("Trials")
            fig_avg_time.set_xlabel("Mean FR (Hz)")
            fig_avg_time.set_title("Mean Firing in Trial")
            
            plt.show()


In [ ]:
dat = alldat[11]
n_neurons, region_index, group_index = generate_metadata(dat)
spikes = dat['spks']
for region in range(n_regions-1):
    pcolormesh_with_single_neuron(region,spikes,11,0)

The above results are not too useful. We have plotted the graphs once again, but this time we decided to plot for a single neuron and look at how it evolves over time. This does not seem to give a good indication of activity at all. We shall now be looking at how to plot the graphs based on only averages of all neurons on a trial wise basis.

In [ ]:
session = 10
dat = alldat[session]
spikes = dat['spks']-np.mean(dat['spks_passive'],axis=(1,2))[0]
n_neurons, region_index, group_index = generate_metadata(dat)
trial = 36

print(np.shape(spikes))
for idx, region in enumerate(np.unique(dat['brain_area'])):
    if np.mod(idx,3)==0:
        fig, axs = plt.subplots(3)
        axs[0].plot(np.mean(spikes[dat['brain_area']==region,trial,:],axis=(0))/dt,label=str(session)+" "+region)
        axs[0].set_xlabel('Time Bins (10ms)')
        axs[0].set_ylabel('Mean Firing Rate for Trials '+str(trial)+' (Hz)')
        axs[0].legend()
    if np.mod(idx,3)==1:
        axs[1].plot(np.mean(spikes[dat['brain_area']==region,trial,:],axis=(0))/dt,label=str(session)+" "+region)
        axs[1].set_xlabel('Time Bins (10ms)')
        axs[1].set_ylabel('Mean Firing Rate for Trials '+str(trial)+' (Hz)')
        axs[1].legend()
    if np.mod(idx,3)==2:
        axs[2].plot(np.mean(spikes[dat['brain_area']==region,trial,:],axis=(0))/dt,label=str(session)+" "+region)
        axs[2].set_xlabel('Time Bins (10ms)')
        axs[2].set_ylabel('Mean Firing Rate for Trials '+str(trial)+' (Hz)')
        axs[2].legend()
        plt.show()

As we can see despite trying to plot the events trial wise, they don't seem to look very nice. But as looks can be decieving we shall try to run a cross correlation anyway.

The correlation that we shall run will be a simple one.

In [ ]:
session = 0
dat = alldat[session]
spikes = dat['spks']-np.mean(dat['spks_passive'],axis=(1,2))[0]
n_neurons, region_index, group_index = generate_metadata(dat)
n_trials = np.shape(spikes)[1]
avg_corrcoef = np.zeros((len(np.unique(dat['brain_area'])),len(np.unique(dat['brain_area']))))
for trial in range(n_trials):
    avg_area_response = avg_area_response = np.zeros((len(np.unique(dat['brain_area'])),np.shape(spikes)[2]))
    for idx, region in enumerate(np.unique(dat['brain_area'])):
        avg_area_response[idx,:]=np.mean(spikes[dat['brain_area']==region,trial,:],axis=(0))/dt
    avg_corrcoef += np.corrcoef(avg_area_response)

avg_corrcoef/=n_trials+1
fig, ax = plt.subplots(1,1)
c=ax.pcolor(avg_corrcoef)
plt.colorbar(c)
ax.set_xticks(ax.get_xticks()[:-1]+0.5)
ax.set_yticks(ax.get_yticks()[:-1]+0.5)
ax.set_xticklabels(np.unique(dat['brain_area']))
ax.set_yticklabels(np.unique(dat['brain_area']))
plt.title("Corrcoef of regions for session "+str(session)+" across all trials.")
plt.show()

As you can see, the correlation coefficient is being calculated properly. so I think there is no issue with this. Let us try to plot the same for all sessions.

In [ ]:
for session in range(39):
    dat = alldat[session]
    spikes = dat['spks']-np.mean(dat['spks_passive'],axis=(1,2))[0]
    n_neurons, region_index, group_index = generate_metadata(dat)
    n_trials = np.shape(spikes)[1]
    avg_corrcoef = np.zeros((len(np.unique(dat['brain_area'])),len(np.unique(dat['brain_area']))))
    for trial in range(n_trials):
        avg_area_response = avg_area_response = np.zeros((len(np.unique(dat['brain_area'])),np.shape(spikes)[2]))
        for idx, region in enumerate(np.unique(dat['brain_area'])):
            avg_area_response[idx,:]=np.mean(spikes[dat['brain_area']==region,trial,:],axis=(0))/dt
        avg_corrcoef += np.corrcoef(avg_area_response)

    avg_corrcoef/=n_trials+1
    fig, ax = plt.subplots(1,1)
    c=ax.pcolor(avg_corrcoef)
    plt.colorbar(c)
    ax.set_xticks(ax.get_xticks()[:-1]+0.5)
    ax.set_yticks(ax.get_yticks()[:-1]+0.5)
    ax.set_xticklabels(np.unique(dat['brain_area']))
    ax.set_yticklabels(np.unique(dat['brain_area']))
    plt.title("Corrcoef of regions for session "+str(session)+" across all trials.")
    plt.show()

In [107]:
import gc
gc.collect()

22